<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/CORINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [ ]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## LUCAS file location

In [ ]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

## Corine file location

In [ ]:
fncorine='/content/drive/MyDrive/CORINE/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif'
Corinefolder=fncorine.replace('/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif','')

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [ ]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install geopandas and pyrpoj for collab

In [ ]:
if colab==1:
  !pip install geopandas
  !pip install pyproj
  !pip install rioxarray

# import libraires

In [ ]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil
import xarray as xr
import rioxarray as rio